In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
from torch.hub import load_state_dict_from_url
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b4
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import efficientnet_v2_s

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#check version
print(torch.__version__)

1.13.1+cu117


In [3]:
PATH_FEAT_TRAIN = 'train_features.csv'
PATH_LABEL_TRAIN = 'train_labels.csv'
RANDOM_SEED = 33
# Load the CSV dataset into a Pandas DataFrame
df_feat = pd.read_csv(PATH_FEAT_TRAIN)
df_label = pd.read_csv(PATH_LABEL_TRAIN)
# Get unique site values from the dataset
unique_sites = df_feat['site'].unique()
print(f'Number of sites: {len(unique_sites)}')

Number of sites: 148


In [4]:
# Extract the one-hot encoded labels as a NumPy array
labels_one_hot = df_label.iloc[:, 1:].values
# Convert one-hot encoded labels to numerical labels
labels_numeric = np.argmax(labels_one_hot, axis=1)
# Add the numerical labels to the DataFrame
df_feat['label'] = labels_numeric

# Split the unique sites into training and val sets
train_sites, val_sites = train_test_split(unique_sites, test_size=0.2, random_state=RANDOM_SEED)

# Split the dataset based on the selected site values
train = df_feat[df_feat['site'].isin(train_sites)]
val = df_feat[df_feat['site'].isin(val_sites)]

# Verify that the 'site' feature is disjoint between the two splits
common_sites = set(train['site']).intersection(set(val['site']))
if len(common_sites) > 0:
    print("Error: The 'site' feature is not disjoint between the splits.")
else:
    print("Splitting successful.")

# Save the training and val sets to separate CSV files
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)

Splitting successful.


In [5]:
labels_numeric

array([1, 6, 1, ..., 3, 1, 3], dtype=int64)

In [6]:
unique_values, counts = np.unique(labels_numeric, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0: 2474
1: 1641
2: 2213
3: 2423
4: 978
5: 2254
6: 2492
7: 2013


In [7]:
val['label'].value_counts()

0    653
6    474
3    388
7    339
2    332
1    263
4    155
5    148
Name: label, dtype: int64

In [8]:
# Save the entire train feat with numerical labels to CSV file
df_feat.to_csv('train_all.csv', index=False)

In [9]:
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "weights")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder has been created successfully!")
else:
    print("successFolder already exists!")

successFolder already exists!


In [10]:
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "submissions")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder has been created successfully!")
else:
    print("successFolder already exists!")

successFolder already exists!


In [11]:
# Set random seeds for CPU
torch.manual_seed(33)
random.seed(33)
np.random.seed(33)

# Set random seeds for GPU (if available)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(33)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [28]:
# Define the path to your CSV file
PATH_CSV_TRAIN = 'train_all.csv'
DIR_IMG = './'
NB_EPOCHS = 4
BATCH_SIZE = 8



# Define the image transformation pipeline for training data
train_transform = transforms.Compose([
    transforms.Resize((366, 366)),  # Resize the input image
    transforms.RandomCrop((320, 320)),  # Randomly crop the image
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ColorJitter(brightness=0.5, contrast=0.4, saturation=0.4, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


# Create a custom dataset class
class CSV_Dataset(Dataset):
    def __init__(self, path_csv_file, transform=None):
        self.data = pd.read_csv(path_csv_file)
        self.transform = transform
        self.classes = sorted(list(self.data['label'].unique()))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(DIR_IMG,self.data.iloc[idx]['filepath'])
        label = self.data.iloc[idx]['label']
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        return image, label

# Create an instance of the custom dataset for training
train_dataset = CSV_Dataset(PATH_CSV_TRAIN, transform=train_transform)

# Create data loaders for training sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

# Load the pre-trained ResNet model
model = efficientnet_b4(weights=EfficientNet_B4_Weights.IMAGENET1K_V1)
#print(model.classifier)
num_classes = len(train_dataset.classes)


# Replace the last fully connected layer with a new one suitable for the number of classes
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features=num_ftrs, out_features=num_classes)


# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# Move the model to the GPU if available
model = model.to(device)


for epoch in range(NB_EPOCHS):
    print(f'Epoch: {epoch}')
    model.train()
    train_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        train_loss += loss.item() * images.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    print(f'Train loss: {train_loss:.5f}') 



path_weights = f'weights/efficientnet_b4_final.pth'
torch.save(model.state_dict(), path_weights)
print(f"Weights saved to {path_weights}!")

Epoch: 0


100%|██████████████████████████████████████████████████████████████████████████████| 2061/2061 [09:12<00:00,  3.73it/s]


Train loss: 1.34959
Epoch: 1


100%|██████████████████████████████████████████████████████████████████████████████| 2061/2061 [09:09<00:00,  3.75it/s]


Train loss: 0.82591
Epoch: 2


100%|██████████████████████████████████████████████████████████████████████████████| 2061/2061 [09:09<00:00,  3.75it/s]


Train loss: 0.63420
Epoch: 3


100%|██████████████████████████████████████████████████████████████████████████████| 2061/2061 [09:24<00:00,  3.65it/s]

Train loss: 0.52254
Weights saved to weights/efficientnet_b4_final.pth!


In [13]:
# Define the path to your CSV file
PATH_CSV_TRAIN = 'train_all.csv'
DIR_IMG = './'
NB_EPOCHS = 2
BATCH_SIZE = 16


# Define the image transformation pipeline for training data
train_transform = transforms.Compose([
    transforms.Resize((366, 366)),  # Resize the input image
    transforms.RandomCrop((320, 320)),  # Randomly crop the image
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ColorJitter(brightness=0.5, contrast=0.4, saturation=0.4, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


# Create a custom dataset class
class CSV_Dataset(Dataset):
    def __init__(self, path_csv_file, transform=None):
        self.data = pd.read_csv(path_csv_file)
        self.transform = transform
        self.classes = sorted(list(self.data['label'].unique()))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(DIR_IMG,self.data.iloc[idx]['filepath'])
        label = self.data.iloc[idx]['label']
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        return image, label

# Create an instance of the custom dataset for training
train_dataset = CSV_Dataset(PATH_CSV_TRAIN, transform=train_transform)


# Create data loaders for training and sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)


# Load the pre-trained ResNet model
model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
num_classes = len(train_dataset.classes)


# Replace the last fully connected layer with a new one suitable for the number of classes
num_ftrs = model.classifier[1].in_features # EfficientNet
model.classifier[1] = nn.Linear(in_features=num_ftrs, out_features=num_classes) # EfficientNet


# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Create a cosine annealing scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# Move the model to the GPU if available
model = model.to(device)


for epoch in range(NB_EPOCHS):
    print(f'Epoch: {epoch}')
    model.train()
    train_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Update the learning rate
        scheduler.step()

        train_loss += loss.item() * images.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    print(f'Train loss: {train_loss:.5f}') 


path_weights = f'weights/efficientnet_v2_s_final.pth'
torch.save(model.state_dict(), path_weights)
print(f"Weights saved to {path_weights}!")

Epoch: 0


100%|██████████████████████████████████████████████████████████████████████████████| 1031/1031 [07:36<00:00,  2.26it/s]


Train loss: 1.14306
Epoch: 1


100%|██████████████████████████████████████████████████████████████████████████████| 1031/1031 [07:35<00:00,  2.26it/s]

Train loss: 0.61587
Weights saved to weights/efficientnet_v2_s_final.pth!


In [14]:
PATH_CSV_VAL = 'test_features.csv'
DIR_IMG = './'
BATCH_SIZE = 64
PATH_WEIGHTS = 'weights/efficientnet_b4_final.pth'

val_transform = transforms.Compose([
    transforms.Resize((384, 384)),  # Resize the input image
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Create a custom dataset class
class CSV_Dataset(Dataset):
    def __init__(self, path_csv_file, transform=None):
        self.data = pd.read_csv(path_csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(DIR_IMG,self.data.iloc[idx]['filepath'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        return image
    
val_dataset = CSV_Dataset(PATH_CSV_VAL, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

# Load the pre-trained ResNet model
model = efficientnet_b4(weights=None)
num_classes = 8

# Replace the last fully connected layer with a new one suitable for the number of classes
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features=num_ftrs, out_features=num_classes)

# Move the model to the GPU if available

model = model.to(device)

model.load_state_dict(torch.load(PATH_WEIGHTS))
# Validation loop
model.eval()

list_labels = []
list_logits = []

with torch.no_grad():
    for images in tqdm(val_loader):
        images = images.to(device)
        outputs = model(images)
        list_logits.append(outputs.cpu().detach().numpy())

arr_logits = np.concatenate(list_logits, axis=0)
probabilities = np.exp(arr_logits) / np.sum(np.exp(arr_logits), axis=1, keepdims=True)

# Specify the file path and name
file_path = "predictions_efficientnet_b4_final.csv"


# Save the NumPy array as CSV
np.savetxt(file_path, probabilities, delimiter=",", fmt="%.18f",
           header="antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent", comments="")

print("CSV file saved successfully.")

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [01:34<00:00,  1.35s/it]

CSV file saved successfully.


In [15]:
PATH_CSV_VAL = 'test_features.csv'
DIR_IMG = './'
BATCH_SIZE = 64
PATH_WEIGHTS = 'weights/efficientnet_v2_s_final.pth'

# Create a custom dataset class
class CSV_Dataset(Dataset):
    def __init__(self, path_csv_file, transform=None):
        self.data = pd.read_csv(path_csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(DIR_IMG, self.data.iloc[idx]['filepath'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        return image

# Assuming you have already defined your val_transform
val_dataset = CSV_Dataset(PATH_CSV_VAL, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

# Load the pre-trained ResNet model
model = efficientnet_v2_s(weights=None)
num_classes = 8

# Replace the last fully connected layer with a new one suitable for the number of classes
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features=num_ftrs, out_features=num_classes)

# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

model.load_state_dict(torch.load(PATH_WEIGHTS))

# Validation loop
model.eval()

list_labels = []
list_logits = []

with torch.no_grad():
    for images in tqdm(val_loader):
        images = images.to(device)
        outputs = model(images)
        list_logits.append(outputs.cpu().detach().numpy())

arr_logits = np.concatenate(list_logits, axis=0)

# Calculate stable probabilities using the Log-Sum-Exp trick
max_logits = np.max(arr_logits, axis=1, keepdims=True)
exp_logits = np.exp(arr_logits - max_logits)
sum_exp_logits = np.sum(exp_logits, axis=1, keepdims=True)
probabilities = exp_logits / sum_exp_logits

# Specify the file path and name
file_path = "predictions_efficientnet_v2_s_final.csv"

# Save the probabilities as a CSV file
column_names = "antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent"
np.savetxt(file_path, probabilities, delimiter=",", fmt="%.18f", header=column_names, comments="")

print("CSV file saved successfully.")

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [01:02<00:00,  1.12it/s]

CSV file saved successfully.


In [16]:
PATH_FEAT_TEST = 'test_features.csv'
PATH_PRED_TEST = 'predictions_efficientnet_v2_s_final.csv'

# Load the CSVs into a Pandas DataFrames
df_feat_test = pd.read_csv(PATH_FEAT_TEST)
df_pred_test = pd.read_csv(PATH_PRED_TEST)

df_concatenated = pd.concat([df_feat_test['id'], df_pred_test], axis=1)

# Save the predictions with 'id' as final submission
df_concatenated.to_csv('submissions/submission_efficientnet_v2_s.csv', index=False, float_format='%.17f')

In [17]:
PATH_PRED_TEST_1 = 'predictions_efficientnet_b4_final.csv'
PATH_PRED_TEST_2 = 'predictions_efficientnet_v2_s_final.csv'

df_pred_test_1 = pd.read_csv(PATH_PRED_TEST_1)
df_pred_test_2 = pd.read_csv(PATH_PRED_TEST_2)

In [18]:
df_pred_test_1

,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,0.117568,0.002528,0.466066,0.146908,0.000955,0.219343,0.017075,0.029557
1,0.759322,0.075972,0.036527,0.007461,0.007243,0.000504,0.051998,0.060973
2,0.874866,0.000366,0.021595,0.037271,0.046623,0.000540,0.006062,0.012677
3,0.000713,0.000050,0.003030,0.000559,0.000292,0.993830,0.000468,0.001058
4,0.400713,0.058642,0.043021,0.004127,0.004579,0.001855,0.438032,0.049031
...,...,...,...,...,...,...,...,...
4459,0.558863,0.146085,0.223677,0.000037,0.012209,0.016060,0.041616,0.001452
4460,0.318218,0.064048,0.257517,0.002539,0.027992,0.044985,0.252139,0.032563
4461,0.352891,0.002252,0.093526,0.066368,0.448201,0.025855,0.004847,0.006058
4462,0.926172,0.008151,0.018517,0.000073,0.008509,0.000557,0.028460,0.009560


In [19]:
df_pred_test_2

,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,0.040666,0.016382,0.336740,0.326252,0.004045,0.028478,0.002185,0.245251
1,0.774848,0.061645,0.043821,0.005129,0.031801,0.002347,0.043639,0.036770
2,0.370690,0.027518,0.041680,0.387572,0.093348,0.002761,0.012677,0.063754
3,0.001153,0.004407,0.004163,0.001578,0.002299,0.983364,0.001959,0.001079
4,0.029922,0.026674,0.009233,0.011672,0.001128,0.001452,0.081906,0.838013
...,...,...,...,...,...,...,...,...
4459,0.204061,0.119244,0.644718,0.000042,0.001436,0.001914,0.028066,0.000520
4460,0.139765,0.015247,0.433451,0.000426,0.005509,0.007690,0.386466,0.011446
4461,0.092884,0.005351,0.658301,0.017711,0.079508,0.011476,0.003082,0.131687
4462,0.745738,0.014403,0.012940,0.000702,0.065390,0.003111,0.147333,0.010384


In [20]:
df_pred_test_2 = df_pred_test_2.dropna()

In [21]:
df_pred_test_2

,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,0.040666,0.016382,0.336740,0.326252,0.004045,0.028478,0.002185,0.245251
1,0.774848,0.061645,0.043821,0.005129,0.031801,0.002347,0.043639,0.036770
2,0.370690,0.027518,0.041680,0.387572,0.093348,0.002761,0.012677,0.063754
3,0.001153,0.004407,0.004163,0.001578,0.002299,0.983364,0.001959,0.001079
4,0.029922,0.026674,0.009233,0.011672,0.001128,0.001452,0.081906,0.838013
...,...,...,...,...,...,...,...,...
4459,0.204061,0.119244,0.644718,0.000042,0.001436,0.001914,0.028066,0.000520
4460,0.139765,0.015247,0.433451,0.000426,0.005509,0.007690,0.386466,0.011446
4461,0.092884,0.005351,0.658301,0.017711,0.079508,0.011476,0.003082,0.131687
4462,0.745738,0.014403,0.012940,0.000702,0.065390,0.003111,0.147333,0.010384


In [22]:
result = (df_pred_test_1 + df_pred_test_2) / 2

In [23]:
result

,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,0.079117,0.009455,0.401403,0.236580,0.002500,0.123911,0.009630,0.137404
1,0.767085,0.068808,0.040174,0.006295,0.019522,0.001426,0.047818,0.048872
2,0.622778,0.013942,0.031638,0.212422,0.069986,0.001650,0.009370,0.038215
3,0.000933,0.002228,0.003597,0.001069,0.001295,0.988597,0.001213,0.001068
4,0.215317,0.042658,0.026127,0.007899,0.002854,0.001654,0.259969,0.443522
...,...,...,...,...,...,...,...,...
4459,0.381462,0.132665,0.434197,0.000039,0.006823,0.008987,0.034841,0.000986
4460,0.228992,0.039647,0.345484,0.001482,0.016751,0.026337,0.319303,0.022004
4461,0.222888,0.003802,0.375914,0.042040,0.263855,0.018666,0.003964,0.068872
4462,0.835955,0.011277,0.015729,0.000387,0.036949,0.001834,0.087896,0.009972


In [24]:
result.to_csv('predictions_ensemble_1.csv', index=False, float_format='%.17f')

In [25]:
df_concatenated = pd.concat([df_feat_test['id'], result], axis=1)

In [26]:
df_concatenated

,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,ZJ016488,0.079117,0.009455,0.401403,0.236580,0.002500,0.123911,0.009630,0.137404
1,ZJ016489,0.767085,0.068808,0.040174,0.006295,0.019522,0.001426,0.047818,0.048872
2,ZJ016490,0.622778,0.013942,0.031638,0.212422,0.069986,0.001650,0.009370,0.038215
3,ZJ016491,0.000933,0.002228,0.003597,0.001069,0.001295,0.988597,0.001213,0.001068
4,ZJ016492,0.215317,0.042658,0.026127,0.007899,0.002854,0.001654,0.259969,0.443522
...,...,...,...,...,...,...,...,...,...
4459,ZJ020947,0.381462,0.132665,0.434197,0.000039,0.006823,0.008987,0.034841,0.000986
4460,ZJ020948,0.228992,0.039647,0.345484,0.001482,0.016751,0.026337,0.319303,0.022004
4461,ZJ020949,0.222888,0.003802,0.375914,0.042040,0.263855,0.018666,0.003964,0.068872
4462,ZJ020950,0.835955,0.011277,0.015729,0.000387,0.036949,0.001834,0.087896,0.009972


In [27]:
df_concatenated.to_csv('submissions/submission_ensemble_v1.csv', index=False, float_format='%.17f')